# 네이버 영화리뷰 감정분석 - SentencePiece

<br><br>

In [2]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
import sentencepiece as spm
from konlpy.tag import Mecab
from tensorflow import keras

<br><br>

아마 여러분들은 네이버 영화리뷰 감정분석 태스크를 한 번쯤은 다루어 보았을 것입니다. 한국어로 된 corpus를 다루어야 하므로 주로 KoNLPy에서 제공하는 형태소 분석기를 사용하여 텍스트를 전처리해서 RNN 모델을 분류기로 사용했을 것입니다.  

만약 이 문제에서 tokenizer를 sentencepiece로 바꾸어 다시 풀어본다면 더 성능이 좋아질까요? 비교해 보는 것도 흥미로울 것입니다.
- 네이버 영화리뷰 감정분석 코퍼스에 sentencepiece를 적용시킨 모델 학습하기
- 학습된 모델로 sp_tokenize() 메소드 구현하기
- 구현된 토크나이저를 적용하여 네이버 영화리뷰 감정분석 모델을 재학습하기
- KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기
- (보너스) SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기
- Word Vector는 활용할 필요가 없습니다. 활용이 가능하지도 않을 것입니다.
- 머지않아 SentencePiece와 BERT 등의 pretrained 모델을 함께 활용하는 태스크를 다루게 될 것입니다.

## 1. 데이터 로드
```
$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt  
$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
```

In [183]:
train_data = pd.read_table('~/aiffel/data/e4_sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/data/e4_sentiment_classification/ratings_test.txt')

In [184]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [185]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [186]:
len(train_data), len(test_data)

(150000, 50000)

<br><br><br><br>

## 2. 데이터 전처리

### 2-1. 중복 및 결측치 제거

In [187]:
# 중복 제거
train_data.drop_duplicates('document', inplace=True)
test_data.drop_duplicates('document', inplace=True)

# 결측치 제거 - axis=0 NaN인 행을 제거
train_data = train_data.dropna()
test_data = test_data.dropna()

In [188]:
len(train_data), len(test_data)

(146182, 49157)

### 2-2. 레이블 분포 확인

In [189]:
train_data.groupby('label').size()

label
0    73342
1    72840
dtype: int64

### 2-3. 한글, 영어, 공백 제외하고 모두 제거

In [190]:
train_data[train_data['id'] == 9976970]['document']

0    아 더빙.. 진짜 짜증나네요 목소리
Name: document, dtype: object

In [191]:
# 한글, 영문, 공백 제외한 나머지 문자 공백으로 치환
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z ]"," ")
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z ]"," ")

In [192]:
train_data[train_data['id'] == 9976970]['document']

0    아 더빙   진짜 짜증나네요 목소리
Name: document, dtype: object

In [193]:
# 다중 공백 제거
train_data['document'] = train_data['document'].str.replace(' +', ' ')
test_data['document'] = test_data['document'].str.replace(' +', ' ')

In [194]:
train_data[train_data['id'] == 9976970]['document']

0    아 더빙 진짜 짜증나네요 목소리
Name: document, dtype: object

In [195]:
# 중복 제거
train_data.drop_duplicates('document', inplace=True)
test_data.drop_duplicates('document', inplace=True)

# 결측치 제거 - axis=0 NaN인 행을 제거
train_data = train_data.dropna()
test_data = test_data.dropna()

In [196]:
len(train_data), len(test_data)

(144871, 48792)

### 2-4. 토큰화 - Sentencepiece 모델 학습
Sentencepiece option
- input : 입력 corpus
- prefix : 저장할 모델 이름
- vocab_size : vocab 개수 (기본 8,000개에 스페셜 토큰 7개를 더해서 8,007개)
- max_sentence_length : 문장의 최대 길이
- pad_id, pad_piece : pad token id, 값
- unk_id, unk_piece : unknown token id, 값
- bos_id, bos_piece : begin of sentence token id, 값
- eos_id, eos_piece : end of sequence token id, 값
- user_defined_symblos : 사용자 정의 토큰

In [81]:
# train_data에서 텍스트 부분만 추출
tmp_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/data/naver.csv'
train_data['document'].to_csv(tmp_file, index=False, header=False)

In [82]:
vocab_size = 8000
model_prefix = 'naver_spm'

spm.SentencePieceTrainer.Train(
    f'--input={tmp_file} --model_prefix={model_prefix} --vocab_size={vocab_size}'
)

### 2-5. 토큰화 - 학습된 모델로 sp_tokenizer() 함수 생성

In [197]:
def sp_tokenize(s, corpus, model_prefix):
    tensor = []
    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))
    
    with open(f'./{model_prefix}.vocab', 'r') as f:
        vocab = f.readlines()
    
    word_index = {}
    index_word = {}
    
    for idx, line in enumerate(vocab):
        word = line.split('\t')[0]
        word_index.update({idx:word})
        index_word.update({word:idx})
    
#     tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, word_index, index_word

In [198]:
s = spm.SentencePieceProcessor()
s.Load(f'{model_prefix}.model')

X_train, word_index, index_word = sp_tokenize(s, train_data['document'], model_prefix)

In [199]:
X_train[0]

[52, 763, 24, 1911, 50, 1648]

In [200]:
X_test = [s.EncodeAsIds(x) for x in test_data['document']]

In [201]:
X_test[0]

[2668, 168]

In [202]:
len(X_train), len(X_test)

(144871, 48792)

### 2-6. 라벨 나눠주기

In [203]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [204]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))
y_train

144871 144871
48792 48792


array([0, 1, 0, ..., 0, 1, 0])

### 2-7. 샘플의 최대 길이 정하기

In [205]:
total_data_text = X_train + X_test
print(len(total_data_text))
num_tokens = [len(token) for token in total_data_text]

193663


In [206]:
print('문장 길이 평균 : ', np.mean(num_tokens))
print('문장 길이 최대 : ', np.max(num_tokens))
print('문장 길이 최소 : ', np.min(num_tokens))
print('문장 길이 표준편차 : ', np.std(num_tokens))

문장 길이 평균 :  15.472578654673324
문장 길이 최대 :  134
문장 길이 최소 :  0
문장 길이 표준편차 :  13.515179433698462


In [207]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)*100}%가 maxlen 설정값 이내에 포함됩니다.')

pad_sequences maxlen :  42
전체 문장의 93.72569876538111%가 maxlen 설정값 이내에 포함됩니다.


In [208]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

144871 48792
144871 48792


In [209]:
tmp_x_train, tmp_y_train = [], []

for i in range(len(X_train)):
    if 0 < len(X_train[i]) < maxlen:
        tmp_x_train.append(X_train[i])
        tmp_y_train.append(y_train[i])

X_train, y_train = tmp_x_train, tmp_y_train

In [210]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

135295 48792
135295 48792


### 2-8. 패딩 설정

In [211]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post')

In [212]:
type(X_train), type(y_train), type(X_test), type(y_test)

(numpy.ndarray, list, numpy.ndarray, numpy.ndarray)

In [213]:
y_train = np.array(y_train)

In [214]:
type(X_train), type(y_train), type(X_test), type(y_test)

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

## 3. 네이버 영화리뷰 감정분석 모델 학습

### 3-1. 모델 구성

In [215]:
word_vector_dim = 16

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.LSTM(8))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          128000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 128,881
Trainable params: 128,881
Non-trainable params: 0
_________________________________________________________________


### 3-2. 모델 학습

In [218]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=512, validation_split=0.2, verbose=1)

Epoch 1/20
212/212 [==============================] - 1s 6ms/step - loss: 0.2371 - accuracy: 0.8819 - val_loss: 0.4408 - val_accuracy: 0.8393
Epoch 2/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2267 - accuracy: 0.8849 - val_loss: 0.4522 - val_accuracy: 0.8368
Epoch 3/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2206 - accuracy: 0.8860 - val_loss: 0.4670 - val_accuracy: 0.8401
Epoch 4/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2138 - accuracy: 0.8887 - val_loss: 0.4564 - val_accuracy: 0.8362
Epoch 5/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2106 - accuracy: 0.8911 - val_loss: 0.4788 - val_accuracy: 0.8333
Epoch 6/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2044 - accuracy: 0.8944 - val_loss: 0.4772 - val_accuracy: 0.8296
Epoch 7/20
212/212 [==============================] - 1s 5ms/step - loss: 0.2006 - accuracy: 0.8975 - val_loss: 0.4795 - val_accuracy: 0.8226
Epoch 

### 3-3. 모델 테스트

In [219]:
results = model.evaluate(X_test,  y_test, verbose=2)

1525/1525 - 4s - loss: 0.6657 - accuracy: 0.8170


<br><br>

## 비교 - KoNLPy 형태소 분석기

In [220]:
len(train_data), len(test_data)

(144871, 48792)

In [221]:
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [222]:
mecab = Mecab()

In [223]:
def mecab_split(sentence):
    return mecab.morphs(sentence)

mecab_corpus = []

for kor in train_data['document']:
    mecab_corpus.append(mecab_split(kor))

In [224]:
mecab_corpus[:4]

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없', '다', '평점', '조정']]

In [225]:
mecab_train_len_list = [len(token) for token in mecab_corpus]
mecab_test_len_list = [len(token) for token in X_test]
mecab_num_tokens = mecab_train_len_list + mecab_test_len_list
print(len(mecab_num_tokens))

193663


In [226]:
print('문장 길이 평균 : ', np.mean(mecab_num_tokens))
print('문장 길이 최대 : ', np.max(mecab_num_tokens))
print('문장 길이 최소 : ', np.min(mecab_num_tokens))
print('문장 길이 표준편차 : ', np.std(mecab_num_tokens))

문장 길이 평균 :  40.47456664411891
문장 길이 최대 :  111
문장 길이 최소 :  0
문장 길이 표준편차 :  42.68243474801159


In [229]:
mecab_max_tokens = np.mean(mecab_num_tokens) + 70
mecab_maxlen = int(mecab_max_tokens)
print('pad_sequences maxlen : ', mecab_maxlen)
print(f'전체 문장의 {np.sum(mecab_num_tokens < mecab_max_tokens) / len(mecab_num_tokens)*100}%가 maxlen 설정값 이내에 포함됩니다.')
# 길이가 111인 문장이 약 25% 정도 되어 따로 삭제하지 않고 그대로 사용한다.

pad_sequences maxlen :  110
전체 문장의 74.80571921327254%가 maxlen 설정값 이내에 포함됩니다.


In [230]:
def tokenize(corpus):  # corpus: Tokenized Sentence's List
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [231]:
# 형태소 기반 토큰화를 진행한 후, 단어 사전의 길이 확인
mecab_tensor, mecab_tokenizer = tokenize(mecab_corpus)
print("MeCab Vocab Size:", len(mecab_tokenizer.index_word))

MeCab Vocab Size: 51210


In [232]:
len(mecab_tensor)

144871

In [234]:
mecab_tensor[0]

array([ 35,  78, 923,  41, 227,  22,  36, 721,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [262]:
tmp_x_test = mecab_tokenizer.texts_to_sequences(test_data['document'])
tmp_x_test = tf.keras.preprocessing.sequence.pad_sequences(tmp_x_test, padding='post')

In [263]:
tmp_x_test[0]

array([809, 132,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [264]:
mecab_x_train = mecab_tensor
mecab_y_train = np.array(train_data['label'])
mecab_x_test = tmp_x_test
mecab_y_test = np.array(test_data['label'])

In [265]:
print(len(mecab_x_train), len(mecab_y_train))
print(len(mecab_x_test), len(mecab_y_test))

144871 144871
48792 48792


In [266]:
print(type(mecab_x_train), type(mecab_y_train))
print(type(mecab_x_test), type(mecab_y_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [269]:
mecab_x_train

array([[   35,    78,   923, ...,     0,     0,     0],
       [ 1006,   500,   513, ...,     0,     0,     0],
       [   26,   204, 29147, ...,     0,     0,     0],
       ...,
       [  152,    90,   194, ...,     0,     0,     0],
       [ 1021,     3,     8, ...,     0,     0,     0],
       [  180,     3,  1890, ...,     0,     0,     0]], dtype=int32)

In [280]:
# check
ch_max = 0
for x in mecab_x_train:
    tmp = max(x)
    if tmp > ch_max:
        ch_max = tmp
print(ch_max)

51210


In [283]:
# 모델 생성
word_vector_dim = 16

model2 = keras.Sequential()
model2.add(keras.layers.Embedding(ch_max+1, word_vector_dim, input_shape=(None,)))
model2.add(keras.layers.LSTM(8))
model2.add(keras.layers.Dense(8, activation='relu'))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 16)          819376    
_________________________________________________________________
lstm_8 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 9         
Total params: 820,257
Trainable params: 820,257
Non-trainable params: 0
_________________________________________________________________


### 3-2. 모델 학습

In [284]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_mecab = model2.fit(mecab_x_train, mecab_y_train, epochs=20, batch_size=512, validation_split=0.2, verbose=1)

Epoch 1/20
227/227 [==============================] - 3s 13ms/step - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 2/20
227/227 [==============================] - 3s 11ms/step - loss: 0.6932 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 3/20
227/227 [==============================] - 3s 11ms/step - loss: 0.6932 - accuracy: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 4/20
227/227 [==============================] - 3s 11ms/step - loss: 0.6932 - accuracy: 0.5010 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 5/20
227/227 [==============================] - 3s 12ms/step - loss: 0.6931 - accuracy: 0.5014 - val_loss: 0.6932 - val_accuracy: 0.4960
Epoch 6/20
227/227 [==============================] - 3s 11ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.5040
Epoch 7/20
227/227 [==============================] - 3s 11ms/step - loss: 0.6931 - accuracy: 0.5013 - val_loss: 0.6931 - val_accuracy: 0.5040

### 3-3. 모델 테스트

In [286]:
results_mecab = model2.evaluate(mecab_x_test,  mecab_y_test, verbose=2)

1525/1525 - 2s - loss: 0.6931 - accuracy: 0.5025


<br><br>

# 루브릭
|평가문항|상세기준|
|:-|:-|
|1. SentencePiece를 이용하여 모델을 만들기까지의 과정이 정상적으로 진행되었는가?|코퍼스 분석, 전처리, SentencePiece 적용, 토크나이저 구현 및 동작이 빠짐없이 진행되었는가?|
|2. SentencePiece를 통해 만든 Tokenizer가 자연어처리 모델과 결합하여 동작하는가?|SentencePiece 토크나이저가 적용된 Text Classifier 모델이 정상적으로 수렴하여 80% 이상의 test accuracy가 확인되었다.|
|3. SentencePiece의 성능을 다각도로 비교분석하였는가?|SentencePiece 토크나이저를 활용했을 때의 성능을 다른 토크나이저 혹은 SentencePiece의 다른 옵션의 경우와 비교하여 분석을 체계적으로 진행하였다.|

## 회고
- 처음에는 전처리 2-3, 2-7을 빼고 진행했을 때는 정확도가 5-60정도 밖에 나오지 않았는데 전처리를 바꾸니 확실히 정확도가 2-30프로 정도 올라갔다. 이렇게 전처리의 중요성을 다시 한 번 느낀다.
- konlpy와 sentencepiece와 비교해보았을 때 확실히 sentencepiece의 정확도가 더 높았었다.
- konlpy(mecab) 공식문서를 확인해봤을 때 vocab size는 따로 지정을 못해주는 것 같다~~(내가 못 찾은 걸 수도)~~. 그래서 모델을 생성할 때 embedding layer의 input_dim을 생성된 vocab size에 맞춰서 넣어주었다. 그래서 사실 sentencepiece와 동일한 상황에서 비교해보고 싶었는데 하지 못한 점이 아쉽다.